## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,85.08,40,72,21.09,broken clouds
1,1,Shingu,JP,33.7333,135.9833,47.70,72,2,3.74,clear sky
2,2,Atuona,PF,-9.8000,-139.0333,78.62,72,6,17.02,clear sky
3,3,Punta Arenas,CL,-53.1500,-70.9167,46.11,65,38,12.35,scattered clouds
4,4,Stromness,GB,58.9650,-3.2960,42.76,56,86,23.44,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for you trip?"))
max_temp = float(input("What is the maximum temperature you would like for you trip?"))

What is the minimum temperature you would like for you trip?70
What is the maximum temperature you would like for you trip?80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & 
                                    (city_data_df["Max Temp"]>=min_temp)]

preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Atuona,PF,-9.8000,-139.0333,78.62,72,6,17.02,clear sky
10,10,Nizwa,OM,22.9333,57.5333,76.37,39,2,6.51,clear sky
15,15,Sisophon,KH,13.5859,102.9737,79.11,50,99,9.60,overcast clouds
25,25,Port Alfred,ZA,-33.5906,26.8910,71.10,91,100,24.09,overcast clouds
32,32,Avarua,CK,-21.2078,-159.7750,78.85,89,20,4.61,few clouds
34,34,Arraial Do Cabo,BR,-22.9661,-42.0278,71.96,83,10,11.86,clear sky
39,39,Atambua,ID,-9.1061,124.8925,77.14,85,100,3.20,overcast clouds
42,42,Najran,SA,17.4924,44.1277,70.65,18,0,4.90,clear sky
44,44,Byron Bay,AU,-28.6500,153.6167,75.15,86,78,12.66,light rain
45,45,Hilo,US,19.7297,-155.0900,70.25,93,90,3.44,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City ID                145
City                   145
Country                145
Lat                    145
Lng                    145
Max Temp               145
Humidity               145
Cloudiness             145
Wind Speed             145
Current Description    145
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name". Used NymPy to create empty cells
import numpy as np
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.62,clear sky,-9.8000,-139.0333,NaN
10,Nizwa,OM,76.37,clear sky,22.9333,57.5333,NaN
15,Sisophon,KH,79.11,overcast clouds,13.5859,102.9737,NaN
25,Port Alfred,ZA,71.10,overcast clouds,-33.5906,26.8910,NaN
32,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,NaN
34,Arraial Do Cabo,BR,71.96,clear sky,-22.9661,-42.0278,NaN
39,Atambua,ID,77.14,overcast clouds,-9.1061,124.8925,NaN
42,Najran,SA,70.65,clear sky,17.4924,44.1277,NaN
44,Byron Bay,AU,75.15,light rain,-28.6500,153.6167,NaN
45,Hilo,US,70.25,overcast clouds,19.7297,-155.0900,NaN


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add location to parameters
    params["location"]=f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print(f'Hotel for {row["City"]} not found... skipping')
        

Hotel for Cusuna not found... skipping
Hotel for Siguiri not found... skipping
Hotel for Goure not found... skipping
Hotel for Tadine not found... skipping
Hotel for Kidal not found... skipping
Hotel for Kerouane not found... skipping
Hotel for Mitu not found... skipping


In [16]:
#check
len(hotel_df)

145

In [17]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             7
dtype: int64

In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.count()

City                   138
Country                138
Max Temp               138
Current Description    138
Lat                    138
Lng                    138
Hotel Name             138
dtype: int64

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig_vaca = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig_vaca.add_layer(marker_layer)

# 11b. Display the figure
fig_vaca

Figure(layout=FigureLayout(height='420px'))